# _**Radiación de Cherenkov**_

El objetivo de este notebook es generar una animación que muestra el fenómeno de radiación de Cherenkov, que ocurre cuando una partícula cargada se mueve a través de un medio dieléctrico a una velocidad mayor que la velocidad de fase de la luz en ese medio. 

### _**Importación de librerias**_

In [ ]:
import meep as mp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import time


### _**Parámetros de simulación**_

Definimos las condiciones de la simulacion tales como el tamaño de celda, las PML layers, la velocidad de nuestra carga , y el indice de refraccion del medio. Posteriormente agregamos el entorno de simulación

In [1]:

## Tamaño de la celda
sx = 60
sy = 60
cell_size = mp.Vector3(sx, sy, 0)

dpml = 1.0
pml_layers = [mp.PML(thickness=dpml)]

v = 1.0  # velocidad de la carga
source_position = mp.Vector3(-0.5 * sx + dpml, 0, 0)  # Initial source position

symmetries = [mp.Mirror(direction=mp.Y)]

sim = mp.Simulation( #Inicio de la simulacion
    resolution=10,
    cell_size=cell_size,
    default_material=mp.Medium(index=1.5),
    symmetries=symmetries,
    boundary_layers=pml_layers,
)

# Ejecutar la visualización
output_file = visualize_simulation(
    sim=sim,
    cell_size=cell_size,
    pml_thickness=dpml,
    source_position_func=move_source,
    max_time=sx/v,
    frame_interval=0.2,
    output_filename='cherenkov_radiation.mp4'
)

# Mostrar el video en el notebook (si estás usando Jupyter)
try:
    from IPython.display import Video
    Video(output_file)
except ImportError:
    print(f"Animación guardada como {output_file}")

Capturando frames cada 0.2 unidades de tiempo...
-----------
Initializing structure...
Halving computational cell along direction y
time for choose_chunkdivision = 0.000657797 s
Working in 2D dimensions.
Computational cell is 60 x 60 x 0 with resolution 10
time for set_epsilon = 0.204004 s
-----------
Meep progress: 23.400000000000002/60.0 = 39.0% done in 4.0s, 6.3s to go
on time step 468 (time=23.4), 0.00858627 s/step
Meep progress: 47.0/60.0 = 78.3% done in 8.0s, 2.2s to go
on time step 940 (time=47), 0.00850144 s/step
run 0 finished at t = 60.0 (1200 timesteps)
Simulación completada en 10.6s
Frames capturados: 300

Creando animación...
Guardando animación como 'cherenkov_radiation.mp4'...
¡Animación guardada con éxito!


### _**Definición de funciones**_

Definimos dos funciones. La funcion de visualizacion usada en notebooks anteriores para ver como se comporta el sistema en el tiempo, y la funcion de mocimiento de carga. Esta funcion tiene como objetivo actulizar continuamente la posicion de la fuente durante la simulacion. Esto adicionado con la baja frecuencia que se le da al parametro source ayuda a emular el comportamiento de una carga puntual.

In [2]:
#Funcion de movimiento de carga
def move_source(sim):
    current_position = mp.Vector3(-0.5 * sx + dpml + v * sim.meep_time(), 0, 0)
    sim.change_sources(
        [
            mp.Source(
                mp.ContinuousSource(frequency=1e-10), ##Frecuencia baja para simula una carga puntual
                component=mp.Ex,
                center=current_position,
            )
        ]
    )
    return current_position
#Funcion de visualizacion
def visualize_simulation(sim, cell_size, pml_thickness, source_position_func, 
                         max_time, frame_interval, output_filename='animation.mp4'):
    """
    Visualiza una simulación de Meep con animación 2D
    
    Parámetros:
    sim: Objeto de simulación Meep
    cell_size: Tamaño de la celda de simulación
    pml_thickness: Grosor de las capas PML
    source_position_func: Función que devuelve la posición actual de la fuente
    max_time: Tiempo máximo de simulación
    frame_interval: Intervalo entre frames capturados
    output_filename: Nombre del archivo de salida para la animación
    """
    
    frames = []
    times = []
    source_positions = []
    
    # Función para capturar frames
    def capture_frame(sim):
        current_source_pos = move_source(sim)
        ez_data = sim.get_array(center=mp.Vector3(), size=mp.Vector3(cell_size.x, cell_size.y, 0), component=mp.Hz)
        frames.append(ez_data.transpose())
        times.append(sim.meep_time())
        source_positions.append(current_source_pos.x)
        return
    
    # Ejecutar simulación para animación
    print(f"Capturando frames cada {frame_interval} unidades de tiempo...")
    start_time = time.time()
    sim.run(
        mp.at_every(frame_interval, capture_frame),
        until=max_time
    )
    print(f"Simulación completada en {time.time()-start_time:.1f}s")
    print(f"Frames capturados: {len(frames)}")
    
    if len(frames) == 0:
        print("Error: No se capturaron frames para la animación")
        return
    
    # Crear animación
    print("\nCreando animación...")
    
    # Calcular rango de color consistente
    all_data = np.concatenate([f.flatten() for f in frames])
    vmax = np.percentile(np.abs(all_data), 99.5) * 1.2
    vmin = -vmax
    
    # Configurar figura
    fig, ax = plt.subplots(figsize=(10, 8))
    extent = [-cell_size.x/2, cell_size.x/2, -cell_size.y/2, cell_size.y/2]
    
    # Mostrar primer frame
    im = ax.imshow(frames[0], cmap='RdBu', vmin=vmin, vmax=vmax,
                  extent=extent, animated=True, aspect='equal')
    
    # Añadir barra de color
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label('Campo Magnético ($H_z$)', fontsize=12)
    
    # Marcar regiones importantes
    pml_boundary = cell_size.x/2 - pml_thickness
    ax.axvline(x=-pml_boundary, color='gray', linestyle='--', alpha=0.7, label='PML Interior')
    ax.axvline(x=pml_boundary, color='gray', linestyle='--', alpha=0.7)
    ax.axhline(y=-pml_boundary, color='gray', linestyle='--', alpha=0.7)
    ax.axhline(y=pml_boundary, color='gray', linestyle='--', alpha=0.7)
    
    # Configuración de título y etiquetas
    ax.set_title(f'Carga en Movimiento (t = {times[0]:.1f})', fontsize=14)
    ax.set_xlabel('Posición X', fontsize=12)
    ax.set_ylabel('Posición Y', fontsize=12)
    
    # Función de actualización para la animación
    def update(frame):
        im.set_array(frames[frame])
        ax.set_title(f'Carga en Movimiento (t = {times[frame]:.1f})', fontsize=14)
        
        # Actualizar posición de la fuente (eliminar línea anterior y dibujar nueva)
        for artist in ax.lines[4:]:  # Eliminar líneas de fuente anteriores
            artist.remove()
        ax.axvline(x=source_positions[frame], color='green', linestyle='-', alpha=0.5, label='Fuente')
        
        return im,
    
    # Crear animación
    anim = FuncAnimation(
        fig, 
        update, 
        frames=len(frames),
        interval=100,  # Tiempo entre frames en ms
        blit=True
    )
    
    plt.close(fig)  # Cerrar figura para ahorrar memoria
    
    # Guardar animación
    print(f"Guardando animación como '{output_filename}'...")
    anim.save(output_filename, writer='ffmpeg', fps=20, dpi=120, bitrate=1800)
    print("¡Animación guardada con éxito!")
    
    return output_filename
